<a href="https://colab.research.google.com/github/AsifYar/Data-Analysis-with-Python/blob/main/GNN_with_TensoeFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spektral

import numpy as np
import tensorflow as tf
import spektral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 129 kB 5.5 MB/s 


In [ ]:
from spektral.datasets import Cora
dataset = Cora()
dataset

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Cora(n_graphs=1)

In [ ]:
adj = dataset[0].a # Adjancy matrix
adj = adj.todense() * np.eye(adj.shape[0]) # Convert sparse matrix to dense and multiply with identity matrix
adj = adj.astype('float32')

features = dataset[0].x
from scipy import sparse
features = sparse.csr_matrix (np.asmatrix(features) )
features = features.todense().astype('float32')


labels = dataset[0].y
train_mask = dataset.mask_tr
test_mask = dataset.mask_te
val_mask = dataset.mask_va

In [ ]:
print(features.shape)
print(adj.shape)
print(labels.shape)

print(np.sum(train_mask)  )
print(np.sum(val_mask)  )
print(np.sum(test_mask)  )

(2708, 1433)
(2708, 2708)
(2708, 7)
140
500
1000


In [ ]:
# define two functions for loss and evaluation

def masked_softmax_cross_entropy (logits, labels , mask): # Returns cross entropy loss by taking only nodes that are masked by mask array
  loss = tf.nn.softmax_cross_entropy_with_logits (logits= logits , labels = labels) # compute loss   ... softmax commonly used for classification problems
  mask = tf.cast(mask , dtype = tf.float32)
  mask /= tf.reduce_mean(mask)
  loss *= mask # apply mask to loss
  return tf.reduce_mean(loss)


def masked_accuracy_matrix(logits , labels , mask): #Compute accuracy matrix for only masked values
     correct_predictions =  tf.equal(tf.argmax(logits , 1) , tf.argmax(labels , 1))
     acc_all = tf.cast(correct_predictions , dtype= tf.float32)
     mask = tf.cast (mask , dtype=tf.float32)
     mask /= tf.reduce_mean(mask)
     acc_all *= mask
     return tf.reduce_mean(acc_all)


In [ ]:
def GNN (fts , adj , transform , activation):
  '''
  fts = feature matrix
  adj = adjancy matrix
  transform = tramsformation we wish to apply to nodes
  activation = activation function
  '''

  seq_fts = transform(fts) # transform origional features
  mul_fts = tf.matmul( adj , seq_fts)
  return activation (mul_fts)


In [ ]:
from tensorflow.python.ops.gen_math_ops import Range
from tensorflow.python.ops.gen_array_ops import identity
# Define a simpe 2 layer GNN to classify cora dataset


def train_cora (fts , adj, gnn_fn, units , epochs , lr , classes , activation_fun):
  lyr_1 = tf.keras.layers.Dense(units=units) # Hidden Layer
  lyr_2 = tf.keras.layers.Dense(classes) # Classification Laer

  def gnn_cora(fts , adj):
    hidden = gnn_fn(fts , adj , lyr_1 , activation_fun) #Compute / activate each unit in hidden layer
    logits = gnn_fn(hidden , adj , lyr_2 , tf.identity)
    return logits

  optmizer = tf.keras.optimizers.Adam(learning_rate=lr)

  # Create a standard training pipeline for tracking
  best_accuracy = 0.0
  for ep in range(epochs):
    with tf.GradientTape() as tape: # Record the gradiants
      logits = gnn_cora(fts , adj)
      loss =   masked_softmax_cross_entropy(logits , labels , train_mask)

    variables = tape.watched_variables()
    grads = tape.gradient(loss , variables)
    optmizer.apply_gradients (zip(grads , variables))

    logits = gnn_cora(fts , adj)
    val_acc = masked_accuracy_matrix(logits , labels , val_mask)
    test_acc = masked_accuracy_matrix(logits , labels , test_mask)

    if (val_acc > best_accuracy):
      best_accuracy = val_acc
      print ('Epochs ' , ep , '|Training loss: ' , loss.numpy() ,' | Validation Accuracy: ' , val_acc.numpy() , ' | Test Accuracy ' , test_acc.numpy()  )

    










In [ ]:
train_cora(features , adj , GNN , 32 , 200 , 0.01 , 7 , tf.nn.relu)

Epochs  0 |Training loss:  3.2374375  | Validation Accuracy:  0.35399997  | Test Accuracy  0.376
Epochs  1 |Training loss:  6.635388  | Validation Accuracy:  0.378  | Test Accuracy  0.37999997
Epochs  2 |Training loss:  3.9438581  | Validation Accuracy:  0.42199996  | Test Accuracy  0.41300002
Epochs  3 |Training loss:  2.3471951  | Validation Accuracy:  0.504  | Test Accuracy  0.552
Epochs  4 |Training loss:  0.95284355  | Validation Accuracy:  0.622  | Test Accuracy  0.6679999
Epochs  5 |Training loss:  0.5943052  | Validation Accuracy:  0.642  | Test Accuracy  0.653
Epochs  18 |Training loss:  0.14604793  | Validation Accuracy:  0.646  | Test Accuracy  0.66
Epochs  19 |Training loss:  0.13292207  | Validation Accuracy:  0.65  | Test Accuracy  0.67199993
Epochs  20 |Training loss:  0.12069238  | Validation Accuracy:  0.66400003  | Test Accuracy  0.67599994
Epochs  21 |Training loss:  0.10927628  | Validation Accuracy:  0.668  | Test Accuracy  0.685
Epochs  22 |Training loss:  0.09948

In [ ]:
train_cora(features , adj , GNN , 32 , 200 , 0.01 , 7 , tf.nn.softmax)

Epochs  0 |Training loss:  1.9795996  | Validation Accuracy:  0.29599997  | Test Accuracy  0.309
Epochs  1 |Training loss:  1.754537  | Validation Accuracy:  0.38799998  | Test Accuracy  0.41400003
Epochs  2 |Training loss:  1.5854099  | Validation Accuracy:  0.466  | Test Accuracy  0.467
Epochs  3 |Training loss:  1.4308763  | Validation Accuracy:  0.574  | Test Accuracy  0.59
Epochs  4 |Training loss:  1.2926052  | Validation Accuracy:  0.62  | Test Accuracy  0.68
Epochs  5 |Training loss:  1.2088951  | Validation Accuracy:  0.642  | Test Accuracy  0.695
Epochs  6 |Training loss:  1.1450729  | Validation Accuracy:  0.662  | Test Accuracy  0.705
Epochs  7 |Training loss:  1.0890396  | Validation Accuracy:  0.666  | Test Accuracy  0.71199995
Epochs  8 |Training loss:  1.0359044  | Validation Accuracy:  0.684  | Test Accuracy  0.71199995
Epochs  9 |Training loss:  0.98747104  | Validation Accuracy:  0.698  | Test Accuracy  0.723
Epochs  10 |Training loss:  0.94308424  | Validation Accur

In [ ]:
train_cora(features , tf.eye(adj.shape[0]) , GNN , 32 , 200 , 0.01 , 7 , tf.nn.relu) # instead of adjancy matrix use 

Epochs  0 |Training loss:  1.9583569  | Validation Accuracy:  0.22999999  | Test Accuracy  0.263
Epochs  1 |Training loss:  1.6807013  | Validation Accuracy:  0.304  | Test Accuracy  0.342
Epochs  2 |Training loss:  1.4382328  | Validation Accuracy:  0.37600002  | Test Accuracy  0.39299995
Epochs  3 |Training loss:  1.1827601  | Validation Accuracy:  0.42799997  | Test Accuracy  0.431
Epochs  4 |Training loss:  0.92878306  | Validation Accuracy:  0.464  | Test Accuracy  0.46299997
Epochs  5 |Training loss:  0.6989178  | Validation Accuracy:  0.48399997  | Test Accuracy  0.49899998
Epochs  6 |Training loss:  0.50805753  | Validation Accuracy:  0.484  | Test Accuracy  0.51299995
Epochs  7 |Training loss:  0.36240962  | Validation Accuracy:  0.486  | Test Accuracy  0.525
Epochs  8 |Training loss:  0.25850803  | Validation Accuracy:  0.508  | Test Accuracy  0.52599996
Epochs  9 |Training loss:  0.18725704  | Validation Accuracy:  0.51799995  | Test Accuracy  0.52199996
Epochs  10 |Training

In [ ]:
degree_matrix = tf.reduce_sum(adj , axis=1)

train_cora(features , adj / degree_matrix , GNN , 32 , 200 , 0.01 , 7 , tf.nn.relu) # instead of adjancy matrix use 

Epochs  0 |Training loss:  1.9375024  | Validation Accuracy:  0.41800004  | Test Accuracy  0.48499995
Epochs  1 |Training loss:  1.7165662  | Validation Accuracy:  0.56  | Test Accuracy  0.589
Epochs  2 |Training loss:  1.473541  | Validation Accuracy:  0.634  | Test Accuracy  0.659
Epochs  3 |Training loss:  1.2152877  | Validation Accuracy:  0.716  | Test Accuracy  0.71199995
Epochs  4 |Training loss:  0.9842669  | Validation Accuracy:  0.766  | Test Accuracy  0.755
Epochs  5 |Training loss:  0.7881317  | Validation Accuracy:  0.77599996  | Test Accuracy  0.77599996
Epochs  6 |Training loss:  0.62600493  | Validation Accuracy:  0.78800005  | Test Accuracy  0.7859999
Epochs  7 |Training loss:  0.49569085  | Validation Accuracy:  0.798  | Test Accuracy  0.782


In [ ]:
# Specific version of normalozation used by Graph Convolution Network

degree_matrix = tf.reduce_sum(adj , axis=1)
normalized_degree = tf.linalg.diag(1.0 / tf.sqrt(degree_matrix))
normalized_adj = tf.matmul(normalized_degree , tf.matmul(adj , normalized_degree ))

train_cora(features , normalized_adj , GNN , 32 , 200 , 0.01 , 7 , tf.nn.relu)

Epochs  0 |Training loss:  1.948059  | Validation Accuracy:  0.406  | Test Accuracy  0.431
Epochs  1 |Training loss:  1.7730652  | Validation Accuracy:  0.54599994  | Test Accuracy  0.54499996
Epochs  2 |Training loss:  1.5704899  | Validation Accuracy:  0.608  | Test Accuracy  0.608
Epochs  3 |Training loss:  1.3479061  | Validation Accuracy:  0.632  | Test Accuracy  0.65599996
Epochs  4 |Training loss:  1.1360527  | Validation Accuracy:  0.69200003  | Test Accuracy  0.71099997
Epochs  5 |Training loss:  0.9407301  | Validation Accuracy:  0.728  | Test Accuracy  0.746
Epochs  6 |Training loss:  0.76528025  | Validation Accuracy:  0.75200003  | Test Accuracy  0.773
Epochs  7 |Training loss:  0.6126938  | Validation Accuracy:  0.77000004  | Test Accuracy  0.79099995
Epochs  8 |Training loss:  0.4854016  | Validation Accuracy:  0.774  | Test Accuracy  0.79399997
Epochs  9 |Training loss:  0.38260373  | Validation Accuracy:  0.77599996  | Test Accuracy  0.798
Epochs  10 |Training loss:  0